## Boto3 API call for QS tables to check refresh time and status

In [18]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-2.0.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
  Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
     |████████████████████████████████| 502 kB 66.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import boto3
quicksight = boto3.client('quicksight')

In [2]:
sts = boto3.client("sts")
account_id = sts.get_caller_identity()["Account"]
print(account_id)

856517911253


In [3]:
import boto3
import pandas as pd

account_id = '[insert_ID]'
quicksight = boto3.client('quicksight', region_name='ap-southeast-1')

all_dataset_summaries = []

response = quicksight.list_data_sets(AwsAccountId=account_id)
all_dataset_summaries.extend(response.get('DataSetSummaries', []))

while 'NextToken' in response:
    response = quicksight.list_data_sets(AwsAccountId=account_id, NextToken=response['NextToken'])
    all_dataset_summaries.extend(response.get('DataSetSummaries', []))

df = pd.DataFrame(all_dataset_summaries)

print(df)
df.to_csv('datasets_prod.csv', index=False)


                                                    Arn  \
0     arn:aws:quicksight:ap-southeast-1:856517911253...   
1     arn:aws:quicksight:ap-southeast-1:856517911253...   
2     arn:aws:quicksight:ap-southeast-1:856517911253...   
3     arn:aws:quicksight:ap-southeast-1:856517911253...   
4     arn:aws:quicksight:ap-southeast-1:856517911253...   
...                                                 ...   
2177  arn:aws:quicksight:ap-southeast-1:856517911253...   
2178  arn:aws:quicksight:ap-southeast-1:856517911253...   
2179  arn:aws:quicksight:ap-southeast-1:856517911253...   
2180  arn:aws:quicksight:ap-southeast-1:856517911253...   
2181  arn:aws:quicksight:ap-southeast-1:856517911253...   

                                 DataSetId  \
0     003332d2-ead7-458c-925e-9686857bc603   
1     0036642e-1a38-4bc6-99b3-1d74e06ce4b2   
2     009d3bbc-97f1-4e2b-8a43-6bd73c53146c   
3     00ec5829-a736-496f-9c29-b74f85c4483a   
4     01089d37-d117-4899-93a5-78c21f918ff2   
...            

In [4]:
account_id = '[insert_ID]'
quicksight = boto3.client('quicksight', region_name='ap-southeast-1')

all_dataset_summaries = []

# List datasets
response = quicksight.list_data_sets(AwsAccountId=account_id)
all_dataset_summaries.extend(response.get('DataSetSummaries', []))

while 'NextToken' in response:
    response = quicksight.list_data_sets(AwsAccountId=account_id, NextToken=response['NextToken'])
    all_dataset_summaries.extend(response.get('DataSetSummaries', []))

all_refresh_schedules = []

# For each dataset, list its refresh schedules
for summary in all_dataset_summaries:
    dataset_id = summary['DataSetId']
    
    try:
        schedules_response = quicksight.list_refresh_schedules(AwsAccountId=account_id, DataSetId=dataset_id)
        refresh_schedules = schedules_response.get('RefreshSchedules', [])
        
        for schedule in refresh_schedules:
            schedule_info = {
                'DataSetId': dataset_id,
                'DataSetName': summary.get('Name', ''),
                'ScheduleId': schedule.get('ScheduleId', ''),
                'Interval': schedule.get('ScheduleFrequency', {}).get('Interval', ''),
                'DayOfWeek': schedule.get('ScheduleFrequency', {}).get('RefreshOnDay', {}).get('DayOfWeek', ''),
                'DayOfMonth': schedule.get('ScheduleFrequency', {}).get('RefreshOnDay', {}).get('DayOfMonth', ''),
                'Timezone': schedule.get('ScheduleFrequency', {}).get('Timezone', ''),
                'TimeOfTheDay': schedule.get('ScheduleFrequency', {}).get('TimeOfTheDay', ''),
                'RefreshType': schedule.get('RefreshType', ''),
                'Arn': schedule.get('Arn', '')
            }
            all_refresh_schedules.append(schedule_info)
    
    except Exception as e:
        print(f"Error retrieving refresh schedules for dataset {dataset_id}: {str(e)}")
        continue

# Convert to pandas DataFrame
df = pd.DataFrame(all_refresh_schedules)

# Save the dataframe to CSV
df.to_csv('refresh_schedules.csv', index=False)


In [6]:
df = pd.read_csv('refresh_schedules.csv')
df

,DataSetId,DataSetName,ScheduleId,Interval,DayOfWeek,DayOfMonth,Timezone,TimeOfTheDay,RefreshType,Arn
0,0117ad1d-8ccc-49dc-9c4f-d88ccd06445e,Auto_play_redshift,f9bb8eab-53e4-46dd-a52f-d7e6508fbd4c,DAILY,NaN,NaN,Asia/Calcutta,18:59,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...
1,016a755e-c4b1-4ab9-adeb-829910f0749e,Transactions with state and city v2 - novels [...,b8a688dc-dd4b-4ba3-9f0d-e23256b18867,MONTHLY,NaN,1,Asia/Calcutta,11:00,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...
2,0213bc13-ed27-46eb-a059-2a1bcfb79197,IOS opt in Notifications Experiment,d577e6f6-d3f7-4b78-abf7-6b3aa8ac9820,DAILY,NaN,NaN,Asia/Calcutta,10:00,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...
3,021bc1dc-7b3e-4ff0-b1b0-68fcce317a3f,Buyer Segmentation And Daily Revenue - Redshift,eab6290b-f1c7-49ad-972c-0545abb0537a,DAILY,NaN,NaN,Asia/Calcutta,09:55,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...
4,021be15b-e814-4dfd-87a8-5597108d9e9f,Analytics Graphs: Chapter wise Valid Readers %...,d6126ab3-8f7f-424f-9064-8fac4dbcac38,WEEKLY,SUNDAY,NaN,Asia/Calcutta,17:59,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...
...,...,...,...,...,...,...,...,...,...,...
652,fe0e89a5-2113-402c-9dcc-96b5d13fce64,Contest Reactivation Inactive Extended Funnel ...,a13189bc-dcb2-455e-bb04-0d1912fd3cad,DAILY,NaN,NaN,Asia/Calcutta,14:59,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...
653,fe338a0a-6ff3-49d6-9c75-bb0cd4eac3ab,Analytics Graphs: Daily Active Readers and Ove...,bb61090b-31f1-4036-8b90-ec787247a215,WEEKLY,SUNDAY,NaN,Asia/Calcutta,14:59,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...
654,fe5c4e39-4613-4efb-9271-a7f68d0b80e3,Last Instance Returning User,a4dddbbe-a1a4-42b3-8076-ad0c4d2dbd33,WEEKLY,MONDAY,NaN,Asia/Calcutta,16:59,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...
655,ff0f3344-2ed6-4bce-8852-0ec1e31ca838,Novels Overall Impressions and Reads Redshift,a380c018-6b2a-4c4e-9d82-684c2833c079,WEEKLY,MONDAY,NaN,Asia/Calcutta,19:59,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...


In [7]:

df_refresh_schedules = pd.read_csv("refresh_schedules.csv")
df_datasets_prod = pd.read_csv("datasets_prod.csv")

df_merged = df_datasets_prod.merge(df_refresh_schedules, on="DataSetId", how="outer")

df_merged.to_csv('merged_prod_datasets.csv', index=False)

df = pd.read_csv('merged_prod_datasets.csv')
df

,Arn_x,DataSetId,Name,CreatedTime,LastUpdatedTime,ImportMode,RowLevelPermissionTagConfigurationApplied,ColumnLevelPermissionRulesApplied,DataSetName,ScheduleId,Interval,DayOfWeek,DayOfMonth,Timezone,TimeOfTheDay,RefreshType,Arn_y
0,arn:aws:quicksight:ap-southeast-1:856517911253...,003332d2-ead7-458c-925e-9686857bc603,creator_translator,2022-06-21 07:35:11.236000+00:00,2023-06-27 12:19:00.147000+00:00,SPICE,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,arn:aws:quicksight:ap-southeast-1:856517911253...,0036642e-1a38-4bc6-99b3-1d74e06ce4b2,Reactivation of Contracted Authors - Published...,2023-08-25 09:26:01.870000+00:00,2023-08-29 09:55:53.872000+00:00,SPICE,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,arn:aws:quicksight:ap-southeast-1:856517911253...,009d3bbc-97f1-4e2b-8a43-6bd73c53146c,"Composite Metrics of New Show Payments, Monthl...",2022-10-14 06:14:33.537000+00:00,2023-04-21 08:30:12.324000+00:00,SPICE,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,arn:aws:quicksight:ap-southeast-1:856517911253...,00ec5829-a736-496f-9c29-b74f85c4483a,People Overview,2021-10-08 01:54:37.889000+00:00,2022-08-03 13:00:28.932000+00:00,SPICE,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,arn:aws:quicksight:ap-southeast-1:856517911253...,01089d37-d117-4899-93a5-78c21f918ff2,QS Sheet - Content Payment - Direct Channel Novel,2022-12-14 07:26:27.275000+00:00,2023-08-10 06:53:06.169000+00:00,SPICE,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2204,arn:aws:quicksight:ap-southeast-1:856517911253...,ff80f3b7-5605-4d21-9024-3a8e6209332b,VIP to Coin Migration Version 2,2022-10-21 03:35:49.600000+00:00,2023-05-10 11:03:52.181000+00:00,SPICE,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2205,arn:aws:quicksight:ap-southeast-1:856517911253...,ffb7ab3d-b4b1-4714-bd8f-451be9a3cf27,Experiment Content BB,2021-07-15 10:14:06.194000+00:00,2022-01-04 12:03:51.669000+00:00,SPICE,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2206,arn:aws:quicksight:ap-southeast-1:856517911253...,ffbfa935-4bc7-4cd9-b23f-575e8137be82,"Daily, Weekly, Monthly AppsFlyer Organic/Marke...",2023-08-31 09:12:28.932000+00:00,2023-08-31 09:54:43.691000+00:00,SPICE,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2207,arn:aws:quicksight:ap-southeast-1:856517911253...,ffc3775e-c07c-48bd-8a8f-91fe22c16ff5,last_show_id_locale_wise,2023-07-26 14:20:10.575000+00:00,2023-07-26 14:23:35.954000+00:00,SPICE,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df = pd.read_csv('merged_prod_datasets.csv')


# Filter rows where 'DataSetName' is not empty
df_filtered = df[df['DataSetName'].notna() & (df['DataSetName'] != '')]

# Write the filtered DataFrame back to the same CSV file
df_filtered.to_csv('filtered_prod_prod.csv', index=False)

df = pd.read_csv('filtered_prod_prod.csv')
df

,Arn_x,DataSetId,Name,CreatedTime,LastUpdatedTime,ImportMode,RowLevelPermissionTagConfigurationApplied,ColumnLevelPermissionRulesApplied,DataSetName,ScheduleId,Interval,DayOfWeek,DayOfMonth,Timezone,TimeOfTheDay,RefreshType,Arn_y
0,arn:aws:quicksight:ap-southeast-1:856517911253...,0117ad1d-8ccc-49dc-9c4f-d88ccd06445e,Auto_play_redshift,2023-01-18 07:28:53.524000+00:00,2023-09-10 13:33:09.064000+00:00,SPICE,False,False,Auto_play_redshift,f9bb8eab-53e4-46dd-a52f-d7e6508fbd4c,DAILY,NaN,NaN,Asia/Calcutta,18:59,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...
1,arn:aws:quicksight:ap-southeast-1:856517911253...,016a755e-c4b1-4ab9-adeb-829910f0749e,Transactions with state and city v2 - novels [...,2023-05-25 04:23:38.127000+00:00,2023-09-01 05:31:01.439000+00:00,SPICE,False,False,Transactions with state and city v2 - novels [...,b8a688dc-dd4b-4ba3-9f0d-e23256b18867,MONTHLY,NaN,1,Asia/Calcutta,11:00,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...
2,arn:aws:quicksight:ap-southeast-1:856517911253...,0213bc13-ed27-46eb-a059-2a1bcfb79197,IOS opt in Notifications Experiment,2023-05-08 09:26:56.601000+00:00,2023-09-11 04:56:09.671000+00:00,SPICE,False,False,IOS opt in Notifications Experiment,d577e6f6-d3f7-4b78-abf7-6b3aa8ac9820,DAILY,NaN,NaN,Asia/Calcutta,10:00,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...
3,arn:aws:quicksight:ap-southeast-1:856517911253...,021bc1dc-7b3e-4ff0-b1b0-68fcce317a3f,Buyer Segmentation And Daily Revenue - Redshift,2023-03-22 07:46:05.605000+00:00,2023-09-11 04:26:19.179000+00:00,SPICE,False,False,Buyer Segmentation And Daily Revenue - Redshift,eab6290b-f1c7-49ad-972c-0545abb0537a,DAILY,NaN,NaN,Asia/Calcutta,09:55,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...
4,arn:aws:quicksight:ap-southeast-1:856517911253...,021be15b-e814-4dfd-87a8-5597108d9e9f,Analytics Graphs: Chapter wise Valid Readers %...,2023-04-20 17:11:57.274000+00:00,2023-09-10 12:58:31.997000+00:00,SPICE,False,False,Analytics Graphs: Chapter wise Valid Readers %...,d6126ab3-8f7f-424f-9064-8fac4dbcac38,WEEKLY,SUNDAY,NaN,Asia/Calcutta,17:59,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,arn:aws:quicksight:ap-southeast-1:856517911253...,fe0e89a5-2113-402c-9dcc-96b5d13fce64,Contest Reactivation Inactive Extended Funnel ...,2023-05-29 10:35:08.226000+00:00,2023-09-11 09:38:13.726000+00:00,SPICE,False,False,Contest Reactivation Inactive Extended Funnel ...,a13189bc-dcb2-455e-bb04-0d1912fd3cad,DAILY,NaN,NaN,Asia/Calcutta,14:59,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...
653,arn:aws:quicksight:ap-southeast-1:856517911253...,fe338a0a-6ff3-49d6-9c75-bb0cd4eac3ab,Analytics Graphs: Daily Active Readers and Ove...,2023-04-20 16:51:38.046000+00:00,2023-09-10 09:50:47.979000+00:00,SPICE,False,False,Analytics Graphs: Daily Active Readers and Ove...,bb61090b-31f1-4036-8b90-ec787247a215,WEEKLY,SUNDAY,NaN,Asia/Calcutta,14:59,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...
654,arn:aws:quicksight:ap-southeast-1:856517911253...,fe5c4e39-4613-4efb-9271-a7f68d0b80e3,Last Instance Returning User,2023-05-01 17:21:27.210000+00:00,2023-09-11 11:38:41.670000+00:00,SPICE,False,False,Last Instance Returning User,a4dddbbe-a1a4-42b3-8076-ad0c4d2dbd33,WEEKLY,MONDAY,NaN,Asia/Calcutta,16:59,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...
655,arn:aws:quicksight:ap-southeast-1:856517911253...,ff0f3344-2ed6-4bce-8852-0ec1e31ca838,Novels Overall Impressions and Reads Redshift,2023-02-07 14:54:11.391000+00:00,2023-09-04 14:29:48.414000+00:00,SPICE,False,False,Novels Overall Impressions and Reads Redshift,a380c018-6b2a-4c4e-9d82-684c2833c079,WEEKLY,MONDAY,NaN,Asia/Calcutta,19:59,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...


In [66]:
pip install timedelta

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
# Read the CSV file into a DataFrame
df = pd.read_csv('filtered_prod_prod.csv', parse_dates=['LastUpdatedTime'])

# Convert 'LastUpdatedTime' to IST (UTC +5:30)
df['LastUpdatedTime_IST'] = df['LastUpdatedTime'] + timedelta(hours=5, minutes=30)

# Write the DataFrame with the new column back to the same CSV file
df.to_csv('filtered_prod_prod.csv', index=False)

df = pd.read_csv('filtered_prod_prod.csv')
df


,Arn_x,DataSetId,Name,CreatedTime,LastUpdatedTime,ImportMode,RowLevelPermissionTagConfigurationApplied,ColumnLevelPermissionRulesApplied,DataSetName,ScheduleId,Interval,DayOfWeek,DayOfMonth,Timezone,TimeOfTheDay,RefreshType,Arn_y,LastUpdatedTime_IST
0,arn:aws:quicksight:ap-southeast-1:856517911253...,0117ad1d-8ccc-49dc-9c4f-d88ccd06445e,Auto_play_redshift,2023-01-18 07:28:53.524000+00:00,2023-09-10 13:33:09.064000+00:00,SPICE,False,False,Auto_play_redshift,f9bb8eab-53e4-46dd-a52f-d7e6508fbd4c,DAILY,NaN,NaN,Asia/Calcutta,18:59,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...,2023-09-10 19:03:09.064000+00:00
1,arn:aws:quicksight:ap-southeast-1:856517911253...,016a755e-c4b1-4ab9-adeb-829910f0749e,Transactions with state and city v2 - novels [...,2023-05-25 04:23:38.127000+00:00,2023-09-01 05:31:01.439000+00:00,SPICE,False,False,Transactions with state and city v2 - novels [...,b8a688dc-dd4b-4ba3-9f0d-e23256b18867,MONTHLY,NaN,1,Asia/Calcutta,11:00,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...,2023-09-01 11:01:01.439000+00:00
2,arn:aws:quicksight:ap-southeast-1:856517911253...,0213bc13-ed27-46eb-a059-2a1bcfb79197,IOS opt in Notifications Experiment,2023-05-08 09:26:56.601000+00:00,2023-09-11 04:56:09.671000+00:00,SPICE,False,False,IOS opt in Notifications Experiment,d577e6f6-d3f7-4b78-abf7-6b3aa8ac9820,DAILY,NaN,NaN,Asia/Calcutta,10:00,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...,2023-09-11 10:26:09.671000+00:00
3,arn:aws:quicksight:ap-southeast-1:856517911253...,021bc1dc-7b3e-4ff0-b1b0-68fcce317a3f,Buyer Segmentation And Daily Revenue - Redshift,2023-03-22 07:46:05.605000+00:00,2023-09-11 04:26:19.179000+00:00,SPICE,False,False,Buyer Segmentation And Daily Revenue - Redshift,eab6290b-f1c7-49ad-972c-0545abb0537a,DAILY,NaN,NaN,Asia/Calcutta,09:55,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...,2023-09-11 09:56:19.179000+00:00
4,arn:aws:quicksight:ap-southeast-1:856517911253...,021be15b-e814-4dfd-87a8-5597108d9e9f,Analytics Graphs: Chapter wise Valid Readers %...,2023-04-20 17:11:57.274000+00:00,2023-09-10 12:58:31.997000+00:00,SPICE,False,False,Analytics Graphs: Chapter wise Valid Readers %...,d6126ab3-8f7f-424f-9064-8fac4dbcac38,WEEKLY,SUNDAY,NaN,Asia/Calcutta,17:59,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...,2023-09-10 18:28:31.997000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,arn:aws:quicksight:ap-southeast-1:856517911253...,fe0e89a5-2113-402c-9dcc-96b5d13fce64,Contest Reactivation Inactive Extended Funnel ...,2023-05-29 10:35:08.226000+00:00,2023-09-11 09:38:13.726000+00:00,SPICE,False,False,Contest Reactivation Inactive Extended Funnel ...,a13189bc-dcb2-455e-bb04-0d1912fd3cad,DAILY,NaN,NaN,Asia/Calcutta,14:59,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...,2023-09-11 15:08:13.726000+00:00
653,arn:aws:quicksight:ap-southeast-1:856517911253...,fe338a0a-6ff3-49d6-9c75-bb0cd4eac3ab,Analytics Graphs: Daily Active Readers and Ove...,2023-04-20 16:51:38.046000+00:00,2023-09-10 09:50:47.979000+00:00,SPICE,False,False,Analytics Graphs: Daily Active Readers and Ove...,bb61090b-31f1-4036-8b90-ec787247a215,WEEKLY,SUNDAY,NaN,Asia/Calcutta,14:59,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...,2023-09-10 15:20:47.979000+00:00
654,arn:aws:quicksight:ap-southeast-1:856517911253...,fe5c4e39-4613-4efb-9271-a7f68d0b80e3,Last Instance Returning User,2023-05-01 17:21:27.210000+00:00,2023-09-11 11:38:41.670000+00:00,SPICE,False,False,Last Instance Returning User,a4dddbbe-a1a4-42b3-8076-ad0c4d2dbd33,WEEKLY,MONDAY,NaN,Asia/Calcutta,16:59,FULL_REFRESH,arn:aws:quicksight:ap-southeast-1:856517911253...,2023-09-11 17:08:41.670000+00:00
655,arn:aws:quicksight:ap-southeast-1:856517911253...,ff0f3344-2ed6-4bce-8852-0ec1e31ca838,Novels Overall Impressions and Reads Redshift,2023-02-07 14:54:11.391000+00:00,2023-09-04 14:29:48.414000+00:00,SPICE,False,False,Novels Overall Impressions and Reads Redshift,a380c

In [18]:
# Initialize the QuickSight client
account_id = '[insert_ID]'
quicksight = boto3.client('quicksight', region_name='ap-southeast-1')

# Read the CSV file into a DataFrame
df = pd.read_csv('filtered_prod_prod.csv')

# For each dataset in the DataFrame, describe it to get additional details
for index, row in df.iterrows():
    dataset_id = row['DataSetId']
    
    try:
        # Get details of the dataset
        dataset_response = quicksight.describe_data_set(AwsAccountId=account_id, DataSetId=dataset_id)
        
        # Extract the SQL query from the response
        physical_table_map = dataset_response.get('DataSet', {}).get('PhysicalTableMap', {})
        
        # The PhysicalTableMap can have multiple keys, iterate over them to find CustomSql
        for _, table_info in physical_table_map.items():
            custom_sql_info = table_info.get('CustomSql', {})
            if 'SqlQuery' in custom_sql_info:
                df.at[index, 'Query'] = custom_sql_info['SqlQuery']
                break
        
    except Exception as e:
        print(f"Error retrieving details for dataset {dataset_id}: {str(e)}")
        continue

# Save the DataFrame with the new details back to a CSV file
df.to_csv('filtered_prod_prod_with_query.csv', index=False)

print("Finished processing. 'Query' column has been added.")


Finished processing. 'Query' column has been added.
